# Kaggle API関連のコード集

- コンペ「[Titanic - Machine Learning from Disaster | Kaggle](https://www.kaggle.com/c/titanic)」

## データセットのダウンロードについて

- コンペの［Data］ページ（例：「[Titanic - Machine Learning from Disaster | Kaggle](https://www.kaggle.com/c/titanic/data)）に、下記のようなコマンドが表示されている。

`kaggle competitions download -c titanic`

APIを使うよりも、これをコマンド実行して、ダウンロードされた.zipファイルを自分で管理した方が早い。データセットのダウンロードを自動化する事情がない限りは、APIは基本的に使わないことにする。

### Kaggle APIでデータセットをダウンロードする方法

In [16]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate() 

# データセット内の全ファイルを.zipファイルでダウンロードする方法
# シグネチャ： dataset_download_files(dataset, path=None, force=False, quiet=True, unzip=False)
# - dataset: データセットを識別するための文字列で、[オーナー]/[データセット名]の形式で指定する。
# - file_name: データセットの設定ファイル
# - path: 定義されていれば、この場所にダウンロードする
# - force: ファイルが既に存在する場合、ダウンロードを強制する (デフォルトはFalse)
# - quiet: 冗長な出力を抑制する(デフォルトはTrue)
api.dataset_download_files('isshiki/titanic', path='..\input_test')
# ※'c/titanic'のようなコンペのデータセットがダウンロードできないので、実質的に上のコマンドを打つしかない。

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Aug 2021 08:00:31 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'ka_sessionid=15a94ba720e8f790fdd285a902d4d04c; max-age=2626560; path=/, GCLB=CODw7v-x_MbFfA; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'X-Cloud-Trace-Context': '15a94ba720e8f790fdd285a902d4d04c/1318201879124987731;o=1', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/datasets/download/competitions/titanic', 'X-Kaggle-MillisecondsElapsed': '45', 'X-Kaggle-RequestId': '2f2903a1df025a12084fd79fcbdf89cf', 'X-Kaggle-ApiVersion': '1.5.12', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-LNwavbr0czCfvpSCmEK85A==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; frame-src 'self' https://www.

ローカル環境とKaggleノートブック環境のPathの違いを吸収する共通コード
--------------------------------------------------------------------

In [3]:
# Kaggleカーネルとローカル環境でのディレクトリ構造の違いを吸収する
import os

is_kaggle = 'KAGGLE_URL_BASE' in set(os.environ.keys())
if is_kaggle:
    compe_name = os.listdir(path='../input')[0]
    input_train_csv_path = '../input/' + compe_name + '/train.csv'
    input_test_csv_path = '../input/' + compe_name + '/test.csv'
    output_dir_path = ''
else:
    input_train_csv_path = '../input/train.csv'
    input_test_csv_path = '../input/test.csv'
    output_dir_path = '../output/'

print('input_train_csv_path:',input_train_csv_path)
print('input_test_csv_path:',input_test_csv_path)
print('output_dir_path',output_dir_path)

input_train_csv_path: ../input/train.csv
input_test_csv_path: ../input/test.csv
output_dir_path ../output/


## 共通コード（認証）

認証は最初に1回だけ行う必要がある。

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

## コンペを検索する方法

In [2]:
competitions = api.competitions_list(search='cat', category="playground")
for comp in competitions:
    print(comp.ref, comp.reward, comp.userRank, sep=',')

cat-in-the-dat,Swag,None
cat-in-the-dat-ii,Swag,None
dogs-vs-cats-redux-kernels-edition,Knowledge,None
dogs-vs-cats,Swag,None


## コンペにSubmissionする方法

どうせページを見るのにAPI経由でやる必要は基本的にないと思う。自動生成した結果をまとめて投稿したい場合には便利そう。

In [18]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate() 

# シグネチャ： competition_submit(file_name, message, competition, quiet=False) method of kaggle.api.kaggle_api_extended.KaggleApi instance
competition = 'titanic' # コンペ名はURLと思われるが、データセットダウンロードのコマンド名を参照すればよい
message = 'Submission test via KaggleApi'
api.competition_submit('../input/gender_submission.csv', message, competition)

100%|██████████| 3.18k/3.18k [00:03<00:00, 1.03kB/s]


Successfully submitted to Titanic - Machine Learning from Disaster

コマンドでも以下のようにしてSubmission可能。だが、間違えやすいと思うので、Webページでやった方がいい。

`kaggle competitions submit titanic -f "input/gender_submission.csv" -m "Submission test via KaggleCommand"`

## 参考資料

- [Kaggle/kaggle-api: Official Kaggle API](https://github.com/Kaggle/kaggle-api#api-credentials)
- [パブリックAPIドキュメント| Kaggle](https://www.kaggle.com/docs/api)
- [Kaggle API - The Missing Python Documentation - Techno Whisp](https://technowhisp.com/kaggle-api-python-documentation/)